# CytnxTutorial2
This tutorial is aimed for those who are familiar with using python doing tensor network calculation. Most of the examples are based on: https://www.tensors.net/p-tutorial-2

In [1]:
##### Python
#
import numpy as np
from numpy import linalg as LA
# ensure 'ncon.py' in working dir
from ncon import ncon

##### Cytnx
import sys
sys.path.append("/usr/local/")
import cytnx
from cytnx import cytnx_extension as cyx

In [2]:
##### Ex2.2(a): SVD of matrix
d1 = 10; d2 = 6
A = np.random.rand(d1,d2)
[U,S,Vh] = LA.svd(A, full_matrices=False)
# print(U.shape)
# print(S.shape)
# print(Vh.shape)
# check result
Af = U @ np.diag(S) @ Vh
dA = LA.norm(Af-A) 

####### Using cytnx.Tensor
Act = cytnx.from_numpy(A)
[Sct, Uct, Vhct] = cytnx.linalg.Svd(Act)

Sct = cytnx.linalg.Diag(Sct)
Afct = cytnx.linalg.Matmul(cytnx.linalg.Matmul(Uct, Sct), Vhct)
print(LA.norm(A - Afct))

####### Using cyx.CyTensor1
##[KHW] You can use Svd ! Svd_truncate is for TN application where we want to truncate the vBond dim.
##[YHC] Thanks for informing. I do it at the next block!

Act2 = cytnx.from_numpy(A)
Act2 = cyx.CyTensor(Act2,1)
Sct,Uct,Vhct = cyx.xlinalg.Svd_truncate(Act2, Act2.shape()[1])
Act2 = cyx.Contract(cyx.Contract(Uct, Sct), Vhct)
Act2 = Act2.get_block()
print(LA.norm(A - Act2))

####### Using cyx.CyTensor2
Act3 = cytnx.from_numpy(A)
Act3 = cyx.CyTensor(Act3,1)
Sct,Uct,Vhct = cyx.xlinalg.Svd(Act3)
Act3 = cyx.Contract(cyx.Contract(Uct, Sct), Vhct)
Act3 = Act3.get_block()
print(LA.norm(A - Act3))

TypeError: loop of ufunc does not support argument 0 of type cytnx.cytnx.Tensor which has no callable sqrt method

In [ ]:
##### Ex2.2(b): SVD of tensor
d = 10; A = np.random.rand(d,d,d)
Am = A.reshape(d**2,d)
Um,Sm,Vh = LA.svd(Am,full_matrices=False)
U = Um.reshape(d,d,d); S = np.diag(Sm)
# check result
Af = ncon([U,S,Vh],[[-1,-2,1],[1,2],[2,-3]])
dA = LA.norm(Af-A)
print(dA)

##### Using Cytnx
## First Method
d = 10; Act = cytnx.from_numpy(A)
Amct = Act.reshape(d**2,d)
[Smct,Umct,Vmct] = cytnx.linalg.Svd(Amct)
Uct = Umct.reshape(d,d,d); Sct = cytnx.linalg.Diag(Smct)
Uct = cyx.CyTensor(Uct,1); Sct = cyx.CyTensor(Sct,1); Vhct = cyx.CyTensor(Vmct,1)
Uct.set_labels([-1,-2,1])
Sct.set_labels([1,2])
Vhct.set_labels([2,-3])
# Uct.print_diagram(); Sct.print_diagram(); Vhct.print_diagram()
Afct = cyx.Contract(cyx.Contract(Uct,Sct),Vhct)
# Afct.print_diagram()



# print(type(Afct))
Afct = Afct.get_block()
dA = Afct - Act
print(LA.norm(np.array(dA)))

ASecondMethod = Amct.clone() # Will be used for the second method

In [ ]:
###### Second Method
Amct = cyx.CyTensor(ASecondMethod,1)
# Amct.print_diagram()
Sct,Uct,Vhct = cyx.xlinalg.Svd_truncate(Amct,d)

# Sct.print_diagram(); Uct.print_diagram(), Vhct.print_diagram()
Afct = cyx.Contract(cyx.Contract(Uct,Sct),Vhct)

AfctPrime = Afct.clone()
# Afct.print_diagram()
# AfctPrime.print_diagram()

Afct = Afct.get_block()
Amct = Amct.get_block()
dA = Amct - Afct
print(LA.norm(dA))


In [ ]:
##### Ex2.2(c): spect. decomp. of matrix
d = 10; A = np.random.rand(d,d)
H = 0.5*(A + A.T) #random Hermitian
D,U = LA.eigh(H)
# check result
Hf = U @ np.diag(D) @ U.T
dH = LA.norm(Hf-H)

########### Using Cytnx
A_ct = cytnx.from_numpy(A)
H_ct = 0.5*(A_ct +A_ct.permute(1,0))
D_ct,U_ct = cytnx.linalg.Eigh(H_ct)
D_ct = cytnx.linalg.Diag(D_ct)

UD_ct = cytnx.linalg.Matmul(U_ct,D_ct)
U_ct_T = U_ct.permute(1,0)
U_ct_T = U_ct_T.contiguous()
print(UD_ct.shape())
print(U_ct_T.shape())
Hf_ct = cytnx.linalg.Matmul(UD_ct, U_ct_T)

dH_ct = np.array(Hf_ct - H_ct)

print(LA.norm(dH_ct))
print()

In [ ]:
##### Ex2.2(d): spect. decomp. of tensor
d = 2; A = np.random.rand(d,d,d,d)
H = 0.5*(A + A.transpose(2,3,0,1))
D,U = LA.eigh(H.reshape(d**2,d**2))
U = U.reshape(d,d,d**2)
# check result
Hf = ncon([U,np.diag(D),U],
           [[-1,-2,1],[1,2],[-3,-4,2]])
dH = LA.norm(Hf-H)

##### Using Cytnx 
d = 2; A_ct = cytnx.from_numpy(np.random.rand(d,d,d,d))
# Atemp = A_ct.permute([2,3,0,1])

H_ct = 0.5*(A_ct + A_ct.permute(2,3,0,1))
D_ct,U_ct = cytnx.linalg.Eigh(H_ct.reshape(d**2,d**2))
U_ct = U_ct.reshape(d,d,d**2)
# check result
U_ct = cyx.CyTensor(U_ct, 1)
D_ct = cytnx.linalg.Diag(D_ct)
D_ct = cyx.CyTensor(D_ct,1)
D_ct.set_labels([1,2])
# U_ct.print_diagram()
U_ct.set_labels([-1,-2,1])
U_ct_T = U_ct.clone()
U_ct_T.set_labels([-3,-4,2])

Hf_ct = cyx.Contract(cyx.Contract(U_ct, D_ct), U_ct_T)
Hf_ct = Hf_ct.get_block()
dH_ct = np.array(Hf_ct - H_ct)
print(LA.norm(dH_ct))

In [ ]:
##### Ex2.3(c)
d = 10; A = np.random.rand(10,10,10,10,10)
# frobenus norm
cvec = [k+1 for k in range(np.ndim(A))]
frobA0 = np.sqrt(ncon([A,np.conj(A)],[cvec,cvec]))
# equivalent frobenus norm
frobA1 = np.sqrt(sum(abs(A.flatten())**2))
# also equivalent frobenus norm
frobA2 = LA.norm(A)
print(frobA0 - frobA2)

A_ct = cytnx.from_numpy(A)
A_ct = cyx.CyTensor(A_ct,2)
# A_ct.print_diagram()
A_ct_T = A_ct.clone()
# A_ct_T.print_diagram()

frobA3 = cyx.Contract(A_ct, A_ct_T).item()**0.5
# frobA3.print_diagram()
print(frobA0-frobA3)

In [ ]:
##### Ex2.4(a): SVD
d = 10; A = np.random.rand(d,d,d,d,d)
Um,S,Vhm = LA.svd(A.reshape(d**3,d**2),full_matrices=False)
U = Um.reshape(d,d,d,d**2)
Vh = Vhm.reshape(d**2,d,d)
##### truncation
chi = 80;
Vhtilda = Vh[:chi,:,:]
Stilda = np.diag(S[:chi])
Utilda = U[:,:,:,:chi]
B = ncon([Utilda,Stilda,Vhtilda],[[-1,-2,-3,1],[1,2],[2,-4,-5]])
##### compare
epsAB = LA.norm(A-B) / LA.norm(A)
print(epsAB)

######### Using Cytnx
A_ct = cytnx.from_numpy(np.random.rand(d,d,d,d,d))
A_ct = cyx.CyTensor(A_ct,3)
# Act.print_diagram()
S_ct,Um_ct,Vhm_ct = cyx.xlinalg.Svd_truncate(A_ct, chi)
B_ct = cyx.Contract(cyx.Contract(Um_ct, S_ct), Vhm_ct)

# A_ct.print_diagram() # Those two look the same in 
# B_ct.print_diagram() # the diagram
A_ct = A_ct.get_block()
# print(A_ct.shape()) # However, [1000, 100]
B_ct = B_ct.get_block() # [10,10,10,10]
# print(B_ct.shape())
epsAB_ct = LA.norm(A_ct-B_ct) / LA.norm(A_ct)
print(epsAB_ct)
